# Bước 1: import thư viện


In [ ]:
import pandas as pd #đọc dữ liệu
import numpy as np #xử lý dữ liệu
import matplotlib.pyplot as plt #vẽ biểu đồ
from sklearn.preprocessing import MinMaxScaler #chuẩn hóa dữ liệu
from keras.callbacks import ModelCheckpoint #lưu lại huấn luyện tốt nhất
from keras.callbacks import EarlyStopping #tránh học tủ
from tensorflow.keras.models import load_model #tải mô hình

#các lớp để xây dựng mô hình
from keras.models import Sequential #đầu vào
from keras.layers import LSTM #học phụ thuộc
from keras.layers import Dropout #tránh học tủ
from keras.layers import Dense #đầu ra

#kiểm tra độ chính xác của mô hình
from sklearn.metrics import r2_score #đo mức độ phù hợp
from sklearn.metrics import mean_absolute_error #đo sai số tuyệt đối trung bình
from sklearn.metrics import mean_absolute_percentage_error #đo % sai số tuyệt đối trung bình

In [ ]:
#tạo quyền truy cập với google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # Đọc file CSV thứ nhất
# df1 = pd.read_csv('/content/drive/MyDrive/Kết Quả/Ket_Qua.csv')

# # Đọc file CSV thứ hai
# df2 = pd.read_csv('/content/drive/MyDrive/Kết Quả/mae_results_3.csv')

# # Ghép hai DataFrame lại với nhau
# df_merged = pd.concat([df1, df2], ignore_index=True)

# # Lưu DataFrame đã ghép vào file CSV mới
# df_merged.to_csv('/content/drive/MyDrive/Kết Quả/Ket_Qua.csv', index=True)

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/Kết Quả/Ket_Qua.csv')
# df1 = df.drop(columns=["Unnamed: 0.1"])

In [ ]:
# df1.to_csv('/content/drive/MyDrive/Kết Quả/Ket_Qua.csv', index=True)

In [ ]:
# đọc dữ liệu từ file csv
df = pd.read_csv('/content/drive/MyDrive/dataset/data.csv')

mae_results = []
# Xóa hai dòng "KL" và "Thay đổi %" từ DataFrame dataSet
df = df.drop(columns=["KL", "% Thay đổi","Mở","Cao","Thấp"])

#định dạng cấu trúc thời gian
df["Ngày"] = pd.to_datetime(df.Ngày,format="%d/%m/%Y")

df1 = pd.DataFrame(df,columns=['Ngày','Đóng cửa'])
df1.index = df1.Ngày
df1.drop('Ngày',axis=1,inplace=True)

# #Tiền xử lý dữ liệu
# from matplotlib.dates import YearLocator, DateFormatter, MonthLocator # Thêm MonthLocator vào để sửa lỗi

# Sắp xếp lại dữ liệu theo thứ tự thời gian
# df1 = df.sort_values(by='Ngày')

# Chuyển đổi định dạng các cột giá thành số thực
df1['Đóng cửa'] = df1['Đóng cửa'].str.replace(',', '').astype(float)


df1.shape

(1615, 1)

In [ ]:
#chia tập dữ liệu
data = df1.values
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

#chuẩn hóa dữ liệu
sc = MinMaxScaler(feature_range=(0 , 1))
sc_train = sc.fit_transform(data)


In [ ]:

for argFor in range(11,70,1):

  #tạo vòng lặp các giá trị
  x_train,y_train=[],[]
  for i in range(argFor,len(train_data)):
    x_train.append(sc_train[i-argFor:i,0]) #lấy argFor giá đóng cửa liên tục
    y_train.append(sc_train[i,0]) #lấy ra giá đóng cửa ngày hôm sau

  #xếp dữ liệu thành 1 mảng 2 chiều
  x_train = np.array(x_train)
  y_train = np.array(y_train)

  #xếp lại dữ liệu thành mảng 1 chiều
  x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
  y_train = np.reshape(y_train,(y_train.shape[0],1))

  # Xây dựng và huấn luyện mô hình

  #xây dựng mô hình
  model = Sequential() #tạo lớp mạng cho dữ liệu đầu vào 2 lớp LSTM
  model.add(LSTM(units=128,input_shape=(x_train.shape[1],1),return_sequences=True))
  model.add(LSTM(units=64))
  model.add(Dropout(0.5)) #loại bỏ 1 số đơn vị tránh học tủ (overfitting)
  model.add(Dense(1)) #output đầu ra 1 chiều
  #đo sai số tuyệt đối trung bình có sử dụng trình tối ưu hóa adam
  model.compile(loss='mean_absolute_error',optimizer='adam')

  #Triển khai early_stopping để dừng khi đã hiệu quả
  early_stopping = EarlyStopping(
    monitor='val_loss',  # Theo dõi giá trị loss trên tập validation
    patience=10,         # Số epochs liên tiếp mà loss không cải thiện trước khi dừng
    restore_best_weights=True  # Khôi phục lại trọng số tốt nhất khi dừng
  )


  #huấn luyện mô hình
  save_model = "/content/drive/MyDrive/save_Model/save_model_1_CTR_2014_2024.hdf5"
  best_model = ModelCheckpoint(save_model,monitor='loss',verbose=2,save_best_only=True,mode='auto')
  model.fit(x_train,y_train,epochs=50,batch_size=argFor,verbose=2,callbacks=[best_model,early_stopping])

  #dữ liệu train
  y_train = sc.inverse_transform(y_train) #giá thực
  final_model = load_model("/content/drive/MyDrive/save_Model/save_model_1_CTR_2014_2024.hdf5")
  y_train_predict = final_model.predict(x_train) #dự đoán giá đóng cửa trên tập đã train
  y_train_predict = sc.inverse_transform(y_train_predict) #giá dự đoán

  # Tái sử dụng mô hình

  #xử lý dữ liệu test
  test = df1[len(train_data)-argFor:].values
  test = test.reshape(-1,1)
  sc_test = sc.transform(test)

  x_test = []
  for i in range(argFor,test.shape[0]):
    x_test.append(sc_test[i-argFor:i,0])
  x_test = np.array(x_test)
  x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

  #dữ liệu test
  y_test = data[train_size:] #giá thực
  y_test_predict = final_model.predict(x_test)
  y_test_predict = sc.inverse_transform(y_test_predict) #giá dự đoán

  #lập biểu đồ so sánh
  train_data1 = df1[argFor:train_size]
  test_data1 = df1[train_size:]


  mae_test = mean_absolute_error(y_test, y_test_predict)
  mae_results.append((argFor, mae_test))
#lưu kết quả ra file csv
results_df = pd.DataFrame(mae_results, columns=['argFor', 'MAE'])
results_df.to_csv('/content/drive/MyDrive/Kết Quả/Ket_Qua_MAE_CTR.csv', index=False)

Epoch 1/50

Epoch 1: loss improved from inf to 0.05739, saving model to /content/drive/MyDrive/save_Model/save_model_1_CTR_2014_2024.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


117/117 - 7s - loss: 0.0574 - 7s/epoch - 58ms/step
Epoch 2/50

Epoch 2: loss improved from 0.05739 to 0.04093, saving model to /content/drive/MyDrive/save_Model/save_model_1_CTR_2014_2024.hdf5


117/117 - 2s - loss: 0.0409 - 2s/epoch - 14ms/step
Epoch 3/50

Epoch 3: loss improved from 0.04093 to 0.03901, saving model to /content/drive/MyDrive/save_Model/save_model_1_CTR_2014_2024.hdf5


117/117 - 2s - loss: 0.0390 - 2s/epoch - 14ms/step
Epoch 4/50

Epoch 4: loss improved from 0.03901 to 0.03680, saving model to /content/drive/MyDrive/save_Model/save_model_1_CTR_2014_2024.hdf5


...

18/18 - 5s - loss: 0.0244 - 5s/epoch - 276ms/step
Epoch 47/50

Epoch 47: loss did not improve from 0.02339


18/18 - 4s - loss: 0.0240 - 4s/epoch - 224ms/step
Epoch 48/50

Epoch 48: loss did not improve from 0.02339


18/18 - 5s - loss: 0.0245 - 5s/epoch - 295ms/step
Epoch 49/50

Epoch 49: loss did not improve from 0.02339


18/18 - 3s - loss: 0.0245 - 3s/epoch - 182ms/step
Epoch 50/50

Epoch 50: loss did not improve from 0.02339


18/18 - 3s - loss: 0.0247 - 3s/epoch - 183ms/step
11/11 [==============================] - 1s 73ms/step


In [ ]:
results_df = pd.DataFrame(mae_results, columns=['argFor', 'MAE'])
results_df.to_csv('/content/drive/MyDrive/Kết Quả/Ket_Qua_MAE_VNM.csv', index=False)

In [ ]:
# Lấy ngày kế tiếp sau ngày cuối cùng trong tập dữ liệu để dự đoán
next_date = df['Ngày'].iloc[-1] + pd.Timedelta(days=1)

# Chuyển đổi ngày kế tiếp sang dạng datetime
next_date = pd.to_datetime(next_date)

# Lấy giá trị của ngày cuối cùng trong tập dữ liệu
next_closing_price = np.array([df['Đóng cửa'].iloc[-1]])  # Lấy giá trị đóng cửa của ngày cuối cùng

# Chuẩn hóa giá trị của ngày cuối cùng
next_closing_price_normalized = sc.transform(next_closing_price.reshape(-1, 1))  # Chuyển thành mảng 2D

# Tạo dự đoán cho ngày kế tiếp bằng mô hình đã huấn luyện
x_next = np.array([sc_train[-argFor:, 0]])  # Lấy argFor giá đóng cửa gần nhất
x_next = np.reshape(x_next, (x_next.shape[0], x_next.shape[1], 1))
y_next_predict = final_model.predict(x_next)
y_next_predict = sc.inverse_transform(y_next_predict)

# Thêm dữ liệu dự đoán của ngày kế tiếp vào DataFrame
df_next = pd.DataFrame({'Ngày': [next_date], 'Đóng cửa': [y_next_predict[0][0]]})
df1 = pd.concat([df1, df_next])

# Vẽ biểu đồ mới với dự đoán cho ngày kế tiếp
plt.figure(figsize=(15, 5))
plt.plot(df1['Ngày'], df1['Đóng cửa'], label='Giá thực tế', color='red')
plt.plot(train_data1.index, train_data1['Dự đoán'], label='Giá dự đoán train', color='green')
plt.plot(test_data1.index, test_data1['Dự đoán'], label='Giá dự đoán test', color='blue')
plt.scatter([next_date], [y_next_predict[0][0]], color='orange', label='Dự đoán ngày kế tiếp')
plt.xlabel('Thời gian')
plt.ylabel('Giá đóng cửa (VNĐ)')
plt.title('So sánh giá dự báo và giá thực tế')
plt.legend()
plt.show()

In [ ]:
# Lấy giá trị của ngày cuối cùng trong tập dữ liệu
actual_closing_price = df['Đóng cửa'].iloc[-1]

# Tạo DataFrame so sánh giá dự đoán với giá ngày cuối trong tập dữ liệu
comparison_df = pd.DataFrame({'Ngày': [next_date], 'Giá dự đoán': [y_next_predict[0][0]], 'Giá ngày trước': [actual_closing_price]})

# In ra bảng so sánh
print(comparison_df)